<a href="https://colab.research.google.com/github/rubuntu/Taller_Introduccion_a_Ciencia_de_Datos_IA_e_Ingenieria_de_Datos/blob/main/sesion_11_comparacion_entre_mlp_y_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de Perros y Gatos: Comparación entre MLP y CNN

## Objetivos
- Cargar y preparar el dataset público `microsoft/cats_vs_dogs` desde HuggingFace.
- Implementar un pipeline de preprocesamiento con `torchvision.transforms` para imágenes.
- Construir y entrenar dos arquitecturas distintas:
  - Un **MLP** (perceptrón multicapa) usando imágenes aplanadas.
  - Una **CNN** simple, con convoluciones y pooling.
- Comparar el rendimiento de ambos modelos en términos de **accuracy**.
- Reflexionar sobre las ventajas y desventajas de MLPs y CNNs en problemas de visión.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

## 1. Cargar dataset

In [ ]:
dataset = load_dataset("microsoft/cats_vs_dogs")

# Dividir en 80% train, 20% test
dataset = dataset["train"].train_test_split(test_size=0.2)

print(dataset)

## 2. Transformaciones
- Para el **MLP**: reducimos resolución a 64x64 y aplanamos la imagen.
- Para la **CNN**: usamos imágenes 128x128 en formato RGB.

In [ ]:
transform_mlp = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # aplanar
])

transform_cnn = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.ToTensor(),
])

## 3. Adaptador HuggingFace → PyTorch Dataset

In [ ]:
class CatsDogsDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.data = hf_dataset
        self.transform = transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        img = self.data[idx]["image"]
        label = self.data[idx]["labels"]  # 0 = cat, 1 = dog
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.long)

train_ds_mlp = CatsDogsDataset(dataset["train"], transform=transform_mlp)
test_ds_mlp  = CatsDogsDataset(dataset["test"], transform=transform_mlp)

train_ds_cnn = CatsDogsDataset(dataset["train"], transform=transform_cnn)
test_ds_cnn  = CatsDogsDataset(dataset["test"], transform=transform_cnn)

train_dl_mlp = DataLoader(train_ds_mlp, batch_size=32, shuffle=True)
test_dl_mlp  = DataLoader(test_ds_mlp, batch_size=32)

train_dl_cnn = DataLoader(train_ds_cnn, batch_size=32, shuffle=True)
test_dl_cnn  = DataLoader(test_ds_cnn, batch_size=32)

## 4. Definir Modelos
### 4.1 MLP

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim=64*64*3, hidden=256):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 2)
        )
    def forward(self, x):
        return self.net(x)

mlp_model = MLP().to(device)

### 4.2 CNN

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),   # 64x64

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),   # 32x32

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),   # 16x16
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*16*16, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 2)
        )
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

cnn_model = SimpleCNN().to(device)

## 5. Función de entrenamiento y evaluación

In [ ]:
def train_model(model, train_dl, test_dl, n_epochs=5, lr=1e-3):
    loss_fn = nn.CrossEntropyLoss()
    opt = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            loss = loss_fn(preds, yb)

            opt.zero_grad()
            loss.backward()
            opt.step()
            running_loss += loss.item()

        # Evaluación
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for xb, yb in test_dl:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb).argmax(dim=1)
                correct += (preds == yb).sum().item()
                total += yb.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1}, Loss={running_loss/len(train_dl):.4f}, Val Acc={acc:.4f}")
    return acc

## 6. Entrenamiento de ambos modelos

In [ ]:
print("Entrenando MLP...")
#acc_mlp = train_model(mlp_model, train_dl_mlp, test_dl_mlp, n_epochs=5, lr=1e-3)
acc_mlp = train_model(mlp_model, train_dl_mlp, test_dl_mlp, n_epochs=1, lr=1e-3)

print("\nEntrenando CNN...")
acc_cnn = train_model(cnn_model, train_dl_cnn, test_dl_cnn, n_epochs=1, lr=1e-3)

print("\nResultados finales:")
print(f"Accuracy MLP: {acc_mlp:.4f}")
print(f"Accuracy CNN: {acc_cnn:.4f}")

## 7. Visualización de algunas predicciones con la CNN

In [ ]:
images, labels = next(iter(test_dl_cnn))
images, labels = images.to(device), labels.to(device)
preds = cnn_model(images).argmax(dim=1)

plt.figure(figsize=(12,6))
for i in range(8):
    plt.subplot(2,4,i+1)
    img = images[i].cpu().permute(1,2,0).numpy()
    plt.imshow(img)
    plt.title(f"Real: {labels[i].item()}, Pred: {preds[i].item()}")
    plt.axis("off")
plt.show()

## Preguntas de Discusión

1. ¿Qué diferencias arquitectónicas hay entre un MLP y una CNN?
2. ¿Por qué el MLP necesita aplanar la imagen mientras que la CNN conserva la estructura espacial?
3. ¿En qué tipo de problemas un MLP podría ser suficiente y en cuáles una CNN es claramente superior?
4. ¿Qué rol cumplen las convoluciones y el pooling en la capacidad de generalización de las CNN?
5. ¿Cómo esperas que varíe el accuracy entre el MLP y la CNN en este dataset?
6. ¿Qué técnicas adicionales (data augmentation, regularización, más capas) podrían mejorar todavía más la CNN?
7. ¿Qué limitaciones pueden encontrarse al entrenar CNNs desde cero con datasets relativamente pequeños?

## 💡 Preguntas de Discusión (desarrolladas)

1. **¿Qué diferencias arquitectónicas hay entre un MLP y una CNN?**

   * Un **MLP (Multilayer Perceptron)** conecta cada píxel de la imagen con la siguiente capa de manera densa → todos los píxeles tienen igual importancia y no se aprovecha la estructura espacial.
   * Una **CNN (Convolutional Neural Network)** utiliza capas convolucionales que procesan regiones locales de la imagen con filtros compartidos, detectando bordes, texturas y formas de manera jerárquica.
   * Resultado: la CNN es mucho más eficiente para imágenes porque reutiliza filtros y respeta la estructura espacial, mientras que un MLP escala mal con la resolución.

---

2. **¿Por qué el MLP necesita aplanar la imagen mientras que la CNN conserva la estructura espacial?**

   * El MLP espera vectores como entrada, no tensores 2D o 3D. Por eso, la imagen debe convertirse en un vector largo (aplanado), perdiendo información espacial.
   * La CNN en cambio acepta tensores de 3 dimensiones (canales, alto, ancho), lo que le permite aprender directamente patrones espaciales sin necesidad de destruir esa estructura.

---

3. **¿En qué tipo de problemas un MLP podría ser suficiente y en cuáles una CNN es claramente superior?**

   * **MLP suficiente:** problemas con entradas de baja dimensión (ej. tabulares, texto codificado como embeddings, imágenes muy pequeñas y simples).
   * **CNN superior:** tareas de visión por computadora donde la estructura espacial es crítica (clasificación de imágenes, detección de objetos, segmentación).
   * En datasets como perros vs gatos, la CNN es claramente superior porque los rasgos distintivos (orejas, hocico, bigotes) dependen de relaciones espaciales entre píxeles.

---

4. **¿Qué rol cumplen las convoluciones y el pooling en la capacidad de generalización de las CNN?**

   * **Convoluciones:** permiten detectar patrones locales (bordes, esquinas, texturas) y reutilizar los mismos filtros en toda la imagen → eficiencia y capacidad de reconocer patrones independientemente de su ubicación.
   * **Pooling (ej. max pooling):** reduce la resolución, mantiene la información más relevante y aporta *invarianza traslacional* (el objeto puede moverse un poco y aún se reconoce).
   * Juntos, hacen que la CNN generalice mejor y no se limite a memorizar píxeles exactos.

---

5. **¿Cómo esperas que varíe el accuracy entre el MLP y la CNN en este dataset?**

   * El **MLP** suele alcanzar un accuracy modesto (≈0.55–0.65) porque ignora relaciones espaciales y necesita más datos para generalizar.
   * La **CNN** debería superar al MLP (≈0.70–0.80), incluso siendo pequeña, ya que aprovecha mejor la estructura visual.
   * La diferencia se vuelve aún mayor en datasets más complejos o con más clases.

---

6. **¿Qué técnicas adicionales (data augmentation, regularización, más capas) podrían mejorar todavía más la CNN?**

   * **Data augmentation:** rotaciones, flips, cambios de color → más robustez.
   * **Regularización:** dropout, weight decay para reducir sobreajuste.
   * **Arquitectura más profunda:** más capas convolucionales y filtros.
   * **Batch normalization:** estabiliza y acelera el entrenamiento.
   * **Learning rate scheduling:** ajusta dinámicamente la tasa de aprendizaje para converger mejor.

---

7. **¿Qué limitaciones pueden encontrarse al entrenar CNNs desde cero con datasets relativamente pequeños?**

   * Riesgo de **sobreajuste**: la red aprende a memorizar las imágenes en vez de generalizar.
   * Dificultad para aprender patrones complejos sin muchos ejemplos.
   * Mayor necesidad de regularización y augmentación.
   * Tiempo de entrenamiento más alto en comparación con MLP.
   * En la práctica, cuando el dataset es pequeño, se suele recurrir a **transfer learning**, usando un modelo preentrenado y adaptándolo al problema.

---
